In [1]:
from pathlib import Path
import json

import gtfstk as gt
import pandas as pd
import numpy as np


DATA_DIR = Path('../data')
OUT_DIR = Path('../output')

In [2]:
path = DATA_DIR/'wellington_gtfs_20171016.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.assess_quality()

,indicator,value
0,num_route_short_names_duplicated,0
1,frac_route_short_names_duplicated,0
2,num_stop_time_dists_missing,340007
3,frac_stop_time_dists_missing,1
4,num_direction_ids_missing,0
5,frac_direction_ids_missing,0
6,num_trips_missing_shapes,0
7,frac_trips_missing_shapes,0
8,num_departure_times_missing,0
9,frac_departure_times_missing,0


In [8]:
shapes_g = feed.shapes_to_geojson()
path = Path('../wellington_shapes_20171016.geojson')
with path.open('w') as tgt:
    json.dump(shapes_g, tgt)

In [3]:
"""
If no shapes, use stops only.
If shapes, then add distances to stop times and to shapes
"""
feed = feed.append_dist_to_shapes()
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)


In [23]:
def refine(xs, n):
    """
    Given a strictly increasing Numpy array ``xs`` of r numbers
    0 <= x_1 < x_2 < ... < x_r <= 1 and an integer ``n`` >= 0, 
    insert into the list ``n`` more numbers between x_1 and x_r such that
    resulting list has
    
    1. no repeats
    2. minimal maximum difference between consecutive numbers
    
    Return the resulting list as a NumPy array.
    """
    while n > 0:
        print(xs, n)
        diffs = np.diff(xs)

        # Get indices i, j of biggest diffs d_i > d_j.
        # Using the method at https://stackoverflow.com/a/23734295 for speed.
        indices = np.argpartition(diffs, -2)[-2:]
        i, j = indices[np.argsort(diffs[indices])]
        d_i, d_j = diffs[i], diffs[j]

        # Choose k least such that d_i/(k + 1) <= d_j
        # with the intent of inserting k evenly spaced points 
        # between x_i and x_{i+1}
        k = int(np.ceil(d_i/d_j - 1))

        # Shrink k if necessary so as not to exceed number of remaining points
        k = min(k, n)

        # Insert the k points, updating xs
        xs = np.concatenate([
          xs[:i + 1], 
          [xs[i] + s*d_i/(k + 1) for s in range(1, k + 1)], 
          xs[i + 1:]
          ])
        
        # Update r
        n -= k
        
    return xs

In [24]:
xs = np.array([0, 0.75, 1])
refine(xs, 1)

[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.

[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.

[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.

[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.

[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.

[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.

[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.

[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.75  1.  ] 1
[ 0.    0.

KeyboardInterrupt: 

In [ ]:
    
def get_trip_points(feed, trip_ids, max_sample_points=100):
    """
    """
    # Filter trips to given trip IDs
    t = feed.trips
    t = t[t['trip_id'].isin(trip_ids)].copy()
    
#     # Many trips can have the same shape, so just choose one trip for each shape
#     t = t.groupby('shape_id').agg('first').reset_index()

    # Get shape geometries
    geom_by_shape = feed.build_geometry_by_shape(shape_ids=t.shape_id)

    # Get stops times for the whittled down list of trips
    st = feed.stop_times
    st = st[st['trip_id'].isin(t.trip_id)].copy()
    
    # Join in stop locations
    st = st.merge(t).merge(feed.stops)
    
    n = max_sample_points
    # Build dictionary trip_id -> list of (lon, lat) sample points
    points_by_trip = {}
    for trip_id, group in st.groupby('trip_id'):
        # Scale stop distances to interval [0, 1]
        d = group['shape_dist_traveled'].iat[-1]
        group['shape_dist_traveled'] /= d
        stop_points = group[['stop_lon', 'stop_lat', 'shape_dist_traveled']].values

        k = group.shape[0]
        # Add n - k more points from trip shape in an even fashion
        assert k <= n
        
        delta = 1/(n - 1)
        ticks = [i*delta for i in range(n)]
        stop_bins = np.digitize(group['shape_dist_traveled'].values, ticks) - 1
        added_ticks = [i*delta/2 for i in range(n-1) if i not in stop_bins]
        shape = group.shape_id.iat[0]
        geom = geom_by_shape[shape]
        shape_points = [
          list(geom.interpolate(tick, normalized=True).coords[0]) + [tick]
          for tick in added_ticks]
        points = sorted(stop_points.tolist() + shape_points, key=lambda x: x[2])
        points = [x[:2] for x in points]
        assert len(points) == n
        points_by_trip[trip_id] = points

    
    return points_by_trip

In [7]:
tids = feed.trips['trip_id'].sample(10)
get_trip_points(feed, tids)

/home/araichev/.virtualenvs/snap_bus_routes/lib/python3.5/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


AssertionError: 